# TP 4 : Programmation Dynamique

### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Résolution du problème

In [ ]:
function solveSacADos(price, weight, capacity)
    nbObjet = length(price)
    C0 = zeros(nbObjet)
    for i in 1:nbObjet
        for j in 1:capacity
            C[i][j] = max(C[i-1][j], C[i-1][j-weight[i]] + price[i])
        end
    end
end

In [ ]:
#Test
price, weight, capacity = readKnaptxtInstance("test.opb.txt")
sol = solveSacADos(price, weight, capacity)

println(sol)